In [ ]:
using ProgressMeter, PyCall, PyPlot, Cairo, Images, HDF5, MultivariateStats, Interpolations, Lasso, Distributions, ImageFiltering
using _Data
using  NaNStatistics, Statistics

include("../project_place_cell/functions/func_map.jl")
include("./Decoder_Functions.jl")
include("./Decoder_Pipeline.jl")
np = pyimport("numpy")

In [ ]:
datasets_corner_cue = 
[
    ["20220407_152537", 4, "jen"],
    ["20220406_111526", 9, "jen"],
    ["20220407_090156", 5, "jen"],
    ["20220417_165530", 2, "jen"],
    ["20220406_153842", 9, "jen"],
    ["20220405_171444", 4, "jen"],
    ["20220416_160516", 6, "jen"]
];


datasets_8s = 
[
    ["20230410_115717", 9, "lorenz"],
    ["20230525_130922", 8, "chuyu"],
    ["20230525_174820", 8, "chuyu"],
    ["20230610_204004", 8, "jen"],  
    ["20230611_141304", 4, "drew"],
    ["20230610_142825", 8, "chuyu"]
];

lengths = [90, 90, 90, 89, 90, 90, 90, 90, 90];

In [ ]:
activity_bins = 7
activity_shift = 4
n_pos = 60 #number of bins in long side
long_axis_in_mm = 47 #47 for rectangular, 33 for others
use_amount = 1000
file_name = "_decoding_revision_new.h5"


i=1

experiment_filename = datasets_corner_cue[i][1]
server = datasets_corner_cue[i][2]
experimenter = datasets_corner_cue[i][3]

ds_Lorenz = Dataset(experiment_filename, "lorenz", gethostname() == "roli-$(server)" ? "/data" : "/nfs/data$(server)")
ds_Chuyu = Dataset(experiment_filename, "chuyu", gethostname() == "roli-$(server)" ? "/data" : "/nfs/data$(server)")

ds = Dataset(experiment_filename, experimenter, gethostname() == "roli-$(server)" ? "/data" : "/nfs/data$(server)")


In [ ]:
#rm(joinpath(data_path(ds_Lorenz), "20220406_153842_fig4_data_2000.h5"))

## import

In [ ]:
readdir(path(ds_Chuyu))

In [ ]:

C, heading, img_bg, y_fish, x_offset, x_fish, y_offset = h5open(ds, "behavior.h5"; raw = true) do file
    read(file, "C"),
    read(file, "heading"),
    read(file, "img_bg"),
    read(file, "fish_yolk_y"),
    read(file, "offset_x"),
    read(file, "fish_yolk_x"),
    read(file, "offset_y")
end;

img_bg_end = img_bg[:,:,end]

# orientation-corrected fish location (time binned)
position_file = h5open(joinpath(data_path(ds_Chuyu), "for_place_calculation_chamber_geometry_$(experiment_filename)_n60.h5"))
    chamber_roi = read(position_file,"chamber_roi")
    x_fish_sweep_mean = read(position_file,"x_fish_sweep_mean")
    y_fish_sweep_mean = read(position_file,"y_fish_sweep_mean")
    speed_mm_s = read(position_file, "speed_mm_s")
    loc_digital = read(position_file, "loc_digital")
    x_digital = read(position_file, "x_digital")
    y_digital = read(position_file, "y_digital")
    x_bins = read(position_file, "x_bins")
    y_bins = read(position_file, "y_bins")
close(position_file)

moving_valid = speed_mm_s .> 0.1;
long_axis_in_bins = (maximum(x_digital)-minimum(x_digital)+1)


file = h5open(joinpath(data_path(ds_Chuyu), "neuron_spatial_info_15_$(lengths[i])_chamber_geometry_$(experiment_filename)_sigma1_n60_A_dF.h5"), "r") #_whole #spatial_info_4 done on merged cells
    valid_neurons = HDF5.readmmap(file["valid_neurons"])
    specificity_shuffle_z = HDF5.readmmap(file["specificity_shuffle_z"])
    specificity_population_z = HDF5.readmmap(file["specificity_population_z"])
    specificity = HDF5.readmmap(file["specificity"])
    bool_index = HDF5.readmmap(file["bool_index"])
close(file)

bool_index = BitArray(bool_index)



file = h5open(joinpath(data_path(ds_Chuyu), "NMF_merge.h5"), "r")
    A_dFF = HDF5.readmmap(file["A_dF"])
    #A_dFF = HDF5.readmmap(file["A_dFF"])

    z_all = HDF5.readmmap(file["Z_all"])
    centroid_x_all = HDF5.readmmap(file["X_all"])
    centroid_y_all = HDF5.readmmap(file["Y_all"])

    neuron_label = read(file, "neuron_label");
close(file)

n_neurons = size(A_dFF, 2)
n_sweeps = size(A_dFF, 1)


# import an process OT and OB
file = h5open(joinpath(data_path(ds_Chuyu), "region_roi_bool.h5"))
    region_names = read(file, "region_names")
    region_roi_bool = read(file, "region_roi_bool")
close(file)


tel_index = findall(region_names .== "Telencephalon -")[1];
mes_index = findall(region_names .== "Mesencephalon -")[1];
rhomb_index = findall(region_names .== "Rhombencephalon -")[1];

tectum_index = findall(region_names .== "Mesencephalon - Tectum Stratum Periventriculare")[1];
olfactory_index = findall(region_names .== "Telencephalon - Olfactory Bulb")[1];
olfactory_index_2 = findall(region_names .== "Telencephalon - Olfactory bulb dopaminergic neuron areas")[1];

mask_tel = falses(n_neurons)
mask_mes = falses(n_neurons)
mask_rhomb = falses(n_neurons)

optic_tectum = falses(n_neurons)
olfactory_bulb = falses(n_neurons)
olfactory_bulb_2 = falses(n_neurons)


@showprogress for which_neuron in 1:n_neurons
    mask_tel[which_neuron] = maximum(region_roi_bool[neuron_label.==which_neuron, tel_index])
    mask_mes[which_neuron] = maximum(region_roi_bool[neuron_label.==which_neuron, mes_index])
    mask_rhomb[which_neuron] = maximum(region_roi_bool[neuron_label.==which_neuron, rhomb_index])
    
    optic_tectum[which_neuron] = maximum(region_roi_bool[neuron_label.==which_neuron, tectum_index])
    olfactory_bulb[which_neuron] = maximum(region_roi_bool[neuron_label.==which_neuron, olfactory_index])
    olfactory_bulb_2[which_neuron] = maximum(region_roi_bool[neuron_label.==which_neuron, olfactory_index_2])
end

mask_tel = findall(mask_tel)
mask_mes = findall(mask_mes)
mask_rhomb = findall(mask_rhomb)
optic_tectum = findall(optic_tectum)
olfactory_bulb = findall(olfactory_bulb)
olfactory_bulb_2 = findall(olfactory_bulb_2)


subplot(2,1,1)
scatter(centroid_x_all, centroid_y_all, s=2)
scatter(centroid_x_all[mask_tel], centroid_y_all[mask_tel], s=2)
scatter(centroid_x_all[mask_mes], centroid_y_all[mask_mes], s=2)
scatter(centroid_x_all[mask_rhomb], centroid_y_all[mask_rhomb], s=2)

scatter(centroid_x_all[optic_tectum], centroid_y_all[optic_tectum], s=2)
scatter(centroid_x_all[olfactory_bulb], centroid_y_all[olfactory_bulb], s=2)
scatter(centroid_x_all[olfactory_bulb_2], centroid_y_all[olfactory_bulb_2], s=2)

subplot(2,1,2)
scatter(centroid_x_all, z_all, s=2)
scatter(centroid_x_all[mask_tel], z_all[mask_tel], s=2)
scatter(centroid_x_all[mask_mes], z_all[mask_mes], s=2)
scatter(centroid_x_all[mask_rhomb], z_all[mask_rhomb], s=2)

scatter(centroid_x_all[optic_tectum], z_all[optic_tectum], s=2)
scatter(centroid_x_all[olfactory_bulb], z_all[olfactory_bulb], s=2)
scatter(centroid_x_all[olfactory_bulb_2], z_all[olfactory_bulb_2], s=2)


w = size(img_bg, 1)
l = size(img_bg, 2);

# Define bins
min_x = 0
min_y = 0
max_x = w
max_y = l

bin_interval = maximum([(max_y-min_y+2)/n_pos,(max_x-min_x+2)/n_pos])

x_bins = collect(min_x-1:bin_interval:min_x+bin_interval*(n_pos)+1);
y_bins = collect(min_y-1:bin_interval:min_y+bin_interval*(n_pos)+1);

x_bins_mid = (x_bins[1:end-1]+x_bins[2:end])/2
y_bins_mid = (y_bins[1:end-1]+y_bins[2:end])/2;

# Digitize data, then we just count the number
x_digital2 = numpy.digitize(x_fish_sweep_mean, x_bins)
y_digital2 = numpy.digitize(y_fish_sweep_mean, y_bins);
loc_digital2 = (y_digital2.-1).*n_pos.+x_digital2;


function bin_to_px(x, y, offset=true)
    if offset
        return (x .- 0.5) .* bin_interval .+ (min_x-1), (y .- 0.5) .* bin_interval .+ (min_y-1)
    else
        return (x .- 0.5) .* bin_interval, (y .- 0.5) .* bin_interval
    end
end

function px_to_bin(x, y)
    return 0.5 .+ ((x .- (min_x-1)) ./ bin_interval), 0.5 .+ ((y .- (min_y-1)) ./ bin_interval)
end

x_in_bins, y_in_bins = px_to_bin(x_fish_sweep_mean, y_fish_sweep_mean);


try
mkdir(data_path(ds_Lorenz))    
catch
    println("save path exists")
end

temp = Decoder.get_top_neurons(use_amount, specificity_population_z[mask_tel], specificity_shuffle_z[mask_tel]);
place_candidates_unique = mask_tel[temp];

## test

In [ ]:
include("./Decoder_Functions.jl")
include("./Decoder_Pipeline.jl")

In [ ]:
# run all datasets fast test
activity_bins = 7
activity_shift = 4
n_pos = 90 #number of bins in long side
long_axis_in_mm = 47 #47 for rectangular, 33 for others
use_amount = 1000

include("./Decoder_Functions.jl")
include("./Decoder_Pipeline.jl")

lengths = [68, 72, 62, 48, 47, 62];

errors_all = []

for i in 1:length(datasets_8s)
    

    experiment_filename = datasets_8s[i][1]
    server = datasets_8s[i][2]
    experimenter = datasets_8s[i][3]

    ds_Lorenz = Dataset(experiment_filename, "lorenz", gethostname() == "roli-$(server)" ? "/data" : "/nfs/data$(server)")
    ds_Chuyu = Dataset(experiment_filename, "chuyu", gethostname() == "roli-$(server)" ? "/data" : "/nfs/data$(server)")

    ds = Dataset(experiment_filename, experimenter, gethostname() == "roli-$(server)" ? "/data" : "/nfs/data$(server)")

    
    C, heading, img_bg = h5open(ds, "behavior.h5"; raw = true) do file
        read(file, "C"),
        read(file, "heading"),
        read(file, "img_bg")
    end;

    img_bg_end = img_bg[:,:,end]

    # orientation-corrected fish location (time binned)
    position_file = h5open(joinpath(data_path(ds_Chuyu), "for_place_calculation_chamber_geometry_$(experiment_filename)_n90.h5"))
        chamber_roi = read(position_file,"chamber_roi")
        x_fish_sweep_mean = read(position_file,"x_fish_sweep_mean")
        y_fish_sweep_mean = read(position_file,"y_fish_sweep_mean")
        speed_mm_s = read(position_file, "speed_mm_s")
        loc_digital = read(position_file, "loc_digital")
        x_digital = read(position_file, "x_digital")
        y_digital = read(position_file, "y_digital")
        x_bins = read(position_file, "x_bins")
        y_bins = read(position_file, "y_bins")
    close(position_file)

    moving_valid = speed_mm_s .> 0.1;
    long_axis_in_bins = (maximum(x_digital)-minimum(x_digital)+1)


    file = h5open(joinpath(data_path(ds_Chuyu), "neuron_spatial_info_0_$(lengths[i])_chamber_geometry_$(experiment_filename)_sigma1_n90.h5"), "r") #_whole #spatial_info_4 done on merged cells
        valid_neurons = HDF5.readmmap(file["valid_neurons"])
        place_cell_index = read(file, "place_cell_index")
        specificity_shuffle_z = HDF5.readmmap(file["specificity_shuffle_z"])
        specificity_population_z = HDF5.readmmap(file["specificity_population_z"])
        specificity = HDF5.readmmap(file["specificity"])
        bool_index = HDF5.readmmap(file["bool_index"])
    close(file)

    bool_index = BitArray(bool_index)

    bool_index[1:30*120] .= 0


    file = h5open(joinpath(data_path(ds_Chuyu), "NMF_merge.h5"), "r")
        A_dFF = HDF5.readmmap(file["A_dFF"])

        z_all = HDF5.readmmap(file["Z_all"])
        centroid_x_all = HDF5.readmmap(file["X_all"])
        centroid_y_all = HDF5.readmmap(file["Y_all"])

        neuron_label = read(file, "neuron_label");
    close(file)

    n_neurons = size(A_dFF, 2)
    n_sweeps = size(A_dFF, 1)



    # import an process OT and OB
    file = h5open(joinpath(data_path(ds_Chuyu), "region_roi_bool.h5"))
        region_names = read(file, "region_names")
        region_roi_bool = read(file, "region_roi_bool")
    close(file)


    tel_index = findall(region_names .== "Telencephalon -")[1];
    mask_tel = falses(n_neurons)

    for which_neuron in 1:n_neurons
        mask_tel[which_neuron] = maximum(region_roi_bool[neuron_label.==which_neuron, tel_index])
    end

    mask_tel = findall(mask_tel)



    w = size(img_bg, 1)
    l = size(img_bg, 2);

    # Define bins
    min_x = 0
    min_y = 0
    max_x = w
    max_y = l

    bin_interval = maximum([(max_y-min_y+2)/n_pos,(max_x-min_x+2)/n_pos])

    x_bins = collect(min_x-1:bin_interval:min_x+bin_interval*(n_pos)+1);
    y_bins = collect(min_y-1:bin_interval:min_y+bin_interval*(n_pos)+1);

    x_bins_mid = (x_bins[1:end-1]+x_bins[2:end])/2
    y_bins_mid = (y_bins[1:end-1]+y_bins[2:end])/2;

    # Digitize data, then we just count the number
    x_digital2 = numpy.digitize(x_fish_sweep_mean, x_bins)
    y_digital2 = numpy.digitize(y_fish_sweep_mean, y_bins);
    loc_digital2 = (y_digital2.-1).*n_pos.+x_digital2;


    function bin_to_px(x, y, offset=true)
        if offset
            return (x .- 0.5) .* bin_interval .+ (min_x-1), (y .- 0.5) .* bin_interval .+ (min_y-1)
        else
            return (x .- 0.5) .* bin_interval, (y .- 0.5) .* bin_interval
        end
    end

    function px_to_bin(x, y)
        return 0.5 .+ ((x .- (min_x-1)) ./ bin_interval), 0.5 .+ ((y .- (min_y-1)) ./ bin_interval)
    end

    x_in_bins2, y_in_bins2 = px_to_bin(x_fish_sweep_mean, y_fish_sweep_mean);
    

    
    
    amount = [0, 2, 3, 4, 5, 6, 7, 8, 9, 10, 30]#, 50, 60, 80, 100, 300, 500, 600, 700, 800, 900, 1000]
    long_shift_errors_mm = fill(NaN, length(amount))
    
    @showprogress for i in 1:length(amount)
        shift = amount[i]
        
        predicted, rolling_var, rolling_valid = Decoder.quick_decoder(bool_index, A_dFF[:, place_cell_index], loc_digital2, activity_bins, shift, n_pos) #rolling_decoder
        
        errors = Decoder.get_distance(predicted[:, 1] , x_in_bins2, predicted[:, 2], y_in_bins2)
        
        long_shift_errors_mm[i] = nanmedian(errors)*long_axis_in_mm/long_axis_in_bins
    end
    
    append!(errors_all, [long_shift_errors_mm])
    
end

In [ ]:
errors_all

In [ ]:
for i in errors_all 
plt.plot([0, 2, 3, 4, 5, 6, 7, 8, 9, 10, 30], i)
plt.axhline(20, alpha=0.5)
plt.xlabel("shift")
plt.ylabel("error [mm]")
#plt.xscale("log")
end

In [ ]:
x

In [ ]:
fig, ax = plt.subplots(figsize=(3,3))
x = hcat(errors_all...)

m = nanmean(x, dim=2)
s = nanstd(x, dim=2)

ax.plot([0, 2, 3, 4, 5, 6, 7, 8, 9, 10, 30], m, color="black", linewidth=1)

plt.fill_between([0, 2, 3, 4, 5, 6, 7, 8, 9, 10, 30], m-s, m+s, color="lightgrey", alpha=0.7)


In [ ]:
argmin(m)

### test all datasets

In [ ]:
# run all datasets fast test
activity_bins = 1
activity_shift = 4
n_pos = 60 #number of bins in long side
long_axis_in_mm = 47 #47 for rectangular, 33 for others
use_amount = 1000

include("./Decoder_Functions.jl")
include("./Decoder_Pipeline.jl")

lengths = [90, 90, 90, 89, 90, 90, 90, 90, 90];

errors_all = []

for i in 1:length(datasets_corner_cue)
    

experiment_filename = datasets_corner_cue[i][1]
server = datasets_corner_cue[i][2]
experimenter = datasets_corner_cue[i][3]

ds_Lorenz = Dataset(experiment_filename, "lorenz", gethostname() == "roli-$(server)" ? "/data" : "/nfs/data$(server)")
ds_Chuyu = Dataset(experiment_filename, "chuyu", gethostname() == "roli-$(server)" ? "/data" : "/nfs/data$(server)")

ds = Dataset(experiment_filename, experimenter, gethostname() == "roli-$(server)" ? "/data" : "/nfs/data$(server)")
    
    
    C, heading, img_bg, y_fish, x_offset, x_fish, y_offset = h5open(ds, "behavior.h5"; raw = true) do file
        read(file, "C"),
        read(file, "heading"),
        read(file, "img_bg"),
        read(file, "fish_yolk_y"),
        read(file, "offset_x"),
        read(file, "fish_yolk_x"),
        read(file, "offset_y")
    end;
    
    img_bg_end = img_bg[:,:,end]
    
    # orientation-corrected fish location (time binned)
    position_file = h5open(joinpath(data_path(ds_Chuyu), "for_place_calculation_chamber_geometry_$(experiment_filename)_n60.h5"))
        chamber_roi = read(position_file,"chamber_roi")
        x_fish_sweep_mean = read(position_file,"x_fish_sweep_mean")
        y_fish_sweep_mean = read(position_file,"y_fish_sweep_mean")
        speed_mm_s = read(position_file, "speed_mm_s")
        loc_digital = read(position_file, "loc_digital")
        x_digital = read(position_file, "x_digital")
        y_digital = read(position_file, "y_digital")
        x_bins = read(position_file, "x_bins")
        y_bins = read(position_file, "y_bins")
    close(position_file)

    moving_valid = speed_mm_s .> 0.1;
    long_axis_in_bins = (maximum(x_digital)-minimum(x_digital)+1)


    file = h5open(joinpath(data_path(ds_Chuyu), "neuron_spatial_info_15_$(lengths[i])_chamber_geometry_$(experiment_filename)_sigma1_n60_A_dF.h5"), "r") #_whole #spatial_info_4 done on merged cells
        valid_neurons = HDF5.readmmap(file["valid_neurons"])
        specificity_shuffle_z = HDF5.readmmap(file["specificity_shuffle_z"])
        specificity_population_z = HDF5.readmmap(file["specificity_population_z"])
        specificity = HDF5.readmmap(file["specificity"])
        bool_index = HDF5.readmmap(file["bool_index"])
    close(file)

    bool_index = BitArray(bool_index)

    #bool_index[1:30*120] .= 0


    file = h5open(joinpath(data_path(ds_Chuyu), "NMF_merge.h5"), "r")
        #A_dF = HDF5.readmmap(file["A_dF"])
        A_dFF = HDF5.readmmap(file["A_dF"])

        z_all = HDF5.readmmap(file["Z_all"])
        centroid_x_all = HDF5.readmmap(file["X_all"])
        centroid_y_all = HDF5.readmmap(file["Y_all"])

        neuron_label = read(file, "neuron_label");
    close(file)

    n_neurons = size(A_dFF, 2)
    n_sweeps = size(A_dFF, 1)



    # import an process OT and OB
    file = h5open(joinpath(data_path(ds_Chuyu), "region_roi_bool.h5"))
        region_names = read(file, "region_names")
        region_roi_bool = read(file, "region_roi_bool")
    close(file)


    tel_index = findall(region_names .== "Telencephalon -")[1];
    mask_tel = falses(n_neurons)

    @showprogress for which_neuron in 1:n_neurons
        mask_tel[which_neuron] = maximum(region_roi_bool[neuron_label.==which_neuron, tel_index])
    end

    mask_tel = findall(mask_tel)



    w = size(img_bg, 1)
    l = size(img_bg, 2);

    # Define bins
    # min_x = 0
    # min_y = 0
    # max_x = w
    # max_y = l
    
    min_x = minimum(x_fish_sweep_mean)
    min_y = minimum(y_fish_sweep_mean)
    max_x = maximum(x_fish_sweep_mean)
    max_y = maximum(y_fish_sweep_mean)

    bin_interval = maximum([(max_y-min_y+2)/n_pos,(max_x-min_x+2)/n_pos])

    x_bins = collect(min_x-1:bin_interval:min_x+bin_interval*(n_pos)+1);
    y_bins = collect(min_y-1:bin_interval:min_y+bin_interval*(n_pos)+1);

    x_bins_mid = (x_bins[1:end-1]+x_bins[2:end])/2
    y_bins_mid = (y_bins[1:end-1]+y_bins[2:end])/2;

    # Digitize data, then we just count the number
    x_digital2 = numpy.digitize(x_fish_sweep_mean, x_bins)
    y_digital2 = numpy.digitize(y_fish_sweep_mean, y_bins);
    loc_digital2 = (y_digital2.-1).*n_pos.+x_digital2;


    function bin_to_px(x, y, offset=true)
        if offset
            return (x .- 0.5) .* bin_interval .+ (min_x-1), (y .- 0.5) .* bin_interval .+ (min_y-1)
        else
            return (x .- 0.5) .* bin_interval, (y .- 0.5) .* bin_interval
        end
    end

    function px_to_bin(x, y)
        return 0.5 .+ ((x .- (min_x-1)) ./ bin_interval), 0.5 .+ ((y .- (min_y-1)) ./ bin_interval)
    end

    x_in_bins2, y_in_bins2 = px_to_bin(x_fish_sweep_mean, y_fish_sweep_mean);
    
    temp = Decoder.get_top_neurons(use_amount, specificity_population_z[mask_tel], specificity_shuffle_z[mask_tel]);
    place_candidates_unique = mask_tel[temp];


baseline_mean, baseline_std = Decoder.baseline_error(x_in_bins[1:n_sweeps], x_in_bins[1:n_sweeps], bool_index; sample=10)

append!(errors_all, [baseline_mean * long_axis_in_mm/40])
    
    
#     A_dF_place_cells = A_dFF[:, place_candidates_unique]

#     pred, var, valid = Decoder.rolling_decoder(bool_index, A_dF_place_cells, loc_digital2, activity_bins, activity_shift, n_pos; decoding_window=120, space=120);
#     errors = Decoder.get_distance(pred[:, 1], x_in_bins2, pred[:, 2], y_in_bins2);

#     append!(errors_all, [nanmedian(errors[bool_index]) * long_axis_in_mm/41])

end

In [ ]:
mean(errors_all)

### test one dataset

In [ ]:
min_x = minimum(x_fish_sweep_mean)
min_y = minimum(y_fish_sweep_mean)
max_x = maximum(x_fish_sweep_mean)
max_y = maximum(y_fish_sweep_mean)


bin_interval = maximum([(max_y-min_y+2)/n_pos,(max_x-min_x+2)/n_pos])

x_bins = collect(min_x-1:bin_interval:min_x+bin_interval*(n_pos)+1);
y_bins = collect(min_y-1:bin_interval:min_y+bin_interval*(n_pos)+1);

x_bins_mid = (x_bins[1:end-1]+x_bins[2:end])/2
y_bins_mid = (y_bins[1:end-1]+y_bins[2:end])/2;

# Digitize data, then we just count the number
x_digital2 = numpy.digitize(x_fish_sweep_mean, x_bins)
y_digital2 = numpy.digitize(y_fish_sweep_mean, y_bins);
loc_digital2 = (y_digital2.-1).*n_pos.+x_digital2;


function bin_to_px(x, y, offset=true)
    if offset
        return (x .- 0.5) .* bin_interval .+ (min_x-1), (y .- 0.5) .* bin_interval .+ (min_y-1)
    else
        return (x .- 0.5) .* bin_interval, (y .- 0.5) .* bin_interval
    end
end

function px_to_bin(x, y)
    return 0.5 .+ ((x .- (min_x-1)) ./ bin_interval), 0.5 .+ ((y .- (min_y-1)) ./ bin_interval)
end

x_in_bins2, y_in_bins2 = px_to_bin(x_fish_sweep_mean, y_fish_sweep_mean);

In [ ]:
include("./Decoder_Functions.jl")
include("./Decoder_Pipeline.jl")

In [ ]:
A_dF_place_cells = A_dFF[:, place_candidates_unique]

pred, var, valid = Decoder.rolling_decoder(bool_index, A_dF_place_cells, loc_digital, activity_bins, activity_shift, n_pos; decoding_window=120);
errors = Decoder.get_distance(pred[:, 1], x_in_bins, pred[:, 2], y_in_bins);


In [ ]:
errors = Decoder.get_distance(pred[:, 1], x_in_bins, pred[:, 2], y_in_bins);

print(nanmedian(errors[bool_index]) * long_axis_in_mm/60)

In [ ]:
figure(figsize=(20,4))
plot(x_in_bins)
plot(pred[:, 1])
xlim(1000, 10000)
figure(figsize=(20,4))
plot(y_in_bins)
plot(pred[:, 2])
xlim(1000, 10000)

In [ ]:

all_pc_maps = Decoder.make_basis_maps(A_dF_place_cells, loc_digital, bool_index, n_pos, true)

density_map = Float32.(sum([all_pc_maps[:,:,i] .> nanpctile(all_pc_maps[:,:,i], 95) for i in 1:size(all_pc_maps, 3)]))
density_map[density_map .== 0] .= 1

imshow(density_map', origin="lower"); colorbar()

### independend maps

In [ ]:
temp = Decoder.get_top_neurons(1000, population_z[mask_tel], specifity_z[mask_tel])
place_candidates_10000 = mask_tel[temp];

A_dF_place_cells_10000 = A_dF[:, place_candidates_10000]


all_pc_maps = Decoder.make_basis_maps(A_dF_place_cells_10000, loc_digital, bool_index, n_pos, true)

indep_matrix = ones(size(all_pc_maps, 3), size(all_pc_maps, 3))
@showprogress for i in 1:size(all_pc_maps, 3)
    for j in 1:i
        indep_matrix[i,j] = nansum(all_pc_maps[:,:,i] .* all_pc_maps[:,:,j]) ./ (sqrt(nansum(all_pc_maps[:,:,i].^2) .* nansum(all_pc_maps[:,:,j].^2)))
        indep_matrix[j,i] = indep_matrix[i,j]
    end
end

In [ ]:
neuron_indep_sorted = [argmin(sum(indep_matrix, dims=2))[1]] #set the one that is most indep to all others

for i in 1:size(all_pc_maps, 3)-1
    indep = sum(abs.(indep_matrix[neuron_indep_sorted, :]), dims=1)[1, :]
    sort_indep = sortperm(indep)
    sort_indep = sort_indep[[!(i in neuron_indep_sorted) for i in sort_indep]]
    append!(neuron_indep_sorted, [sort_indep[1]])
end

In [ ]:
combined_sorting = Decoder.get_top_neurons(9000, neuron_indep_sorted, temp)


figure(figsize=(20,5))
for i in 1:30
    
    subplot(3, 10, i)
    imshow(all_pc_maps[:,1:30,neuron_indep_sorted[i]]', origin="lower", interpolation="nearest")
    axis("off")
end

In [ ]:
neurons_chosen = [place_candidates_unique[1]]

left_neurons = copy(place_candidates_unique)


A_dF_candidate_cells = @view A_dF[:, place_candidates_unique[1]]

pred2, var2, valid2 = Decoder.rolling_decoder(bool_index, A_dF_candidate_cells, loc_digital, activity_bins, activity_shift, n_pos; decoding_window=120);
errors2 = Decoder.get_distance(pred2[:, 1], x_in_bins, pred2[:, 2], y_in_bins);

errors = [median(errors2[bool_index])]

@showprogress for i in 1:10 # include more neurons
    
    current_min = Inf
    current_argmin = NaN
    
    @showprogress for n in left_neurons
        A_dF_candidate_cells = @view A_dF[:, cat(neurons_chosen, [n], dims=1)]
        
        pred2, var2, valid2 = Decoder.rolling_decoder(bool_index, A_dF_candidate_cells, loc_digital, activity_bins, activity_shift, n_pos; decoding_window=120);
        errors2 = Decoder.get_distance(pred2[:, 1], x_in_bins, pred2[:, 2], y_in_bins);
        
        if nanmedian(errors2[bool_index]) < current_min
            current_argmin = copy(n)
            current_min = copy(nanmedian(errors2[bool_index]))
        end
    end

    append!(errors, [current_min])
    append!(neurons_chosen, [current_argmin])
end


In [ ]:
plt.plot(1:length(neurons_chosen), errors.*(long_axis_in_mm/n_pos))
plt.xlabel("cells included")
plt.ylabel("error [mm]")
plt.xscale("log"); title("greedy chosing")

In [ ]:
amount = [1,2,3,5,7,10,15,20,30,50,100,200, 500, 1000]

combined_sorting = Decoder.get_top_neurons(9000, neuron_indep_sorted, temp)

place_cells = [place_candidates_unique[combined_sorting[1:x]] for x in amount]


error_by_amount_place_cells = DecoderPlots.prepare_subplot4_5(A_dF[:, valid_neurons], place_cells, bool_index, loc_digital, x_in_bins, y_in_bins, n_pos, activity_bins, activity_shift);

plt.plot(amount, error_by_amount_place_cells.*(long_axis_in_mm/n_pos))
plt.xlabel("cells included")
plt.ylabel("error [mm]")
plt.xscale("log")

## example_decoding

In [ ]:
example_stuff();


## error by populations

In [ ]:
use_cells = 1000

populations = Dict()

n_neurons = size(A_dFF, 2)

populations["random_cells"] = Distributions.sample(intersect(valid_neurons, findall(specificity_population_z .< 1)), use_cells, replace=false)

labels = [
        ["place_cells", valid_neurons],
        ["place_cells_tel", mask_tel],
        ["place_cells_rhomb", mask_rhomb],
        ["place_cells_mes", mask_mes],
        ["place_cells_rhomb_mes", union(mask_mes, mask_rhomb)],
        ["place_cells_optic_tectum", optic_tectum]
]

for l in labels
    place_cells = Decoder.get_top_neurons(use_cells, specificity_population_z[l[2]], specificity_shuffle_z[l[2]])
    populations[l[1]] = l[2][place_cells]
end


In [ ]:
sum(isnan.(specificity_shuffle_z))

In [ ]:
scatter(centroid_x_all, centroid_y_all)
scatter(centroid_x_all[populations["place_cells_rhomb_mes"]], centroid_y_all[populations["place_cells_rhomb_mes"]])
scatter(centroid_x_all[populations["place_cells_tel"]], centroid_y_all[populations["place_cells_tel"]])

In [ ]:
error_by_populations(use_amount, populations)

## error by number figure

In [ ]:
error_by_amount()

In [ ]:
error_by_amount(space_n=240)

## error by number figure greedy

In [ ]:
error_by_amount_greedy(25)

## error by long shift

In [ ]:
error_long_shift()

## decoding animation

In [ ]:
decoding_animation(activity_bins, activity_shift);

## heatmap figure

mean error of different shifts and bins

In [ ]:
error_bin_shift()